# Build the dataset from the SQL database and dump it into a CSV file

## 0. Useful imports

In [6]:
# Import libraries
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

# below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

%matplotlib inline
plt.style.use('ggplot')

## 1. Connection to postgresql

In [7]:
# information used to create a database connection
sqluser = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to postgres with a copy of the MIMIC-III database
con = psycopg2.connect(dbname=dbname, user=sqluser)

# the below statement is prepended to queries to ensure they select from the right schema
query_schema = 'set search_path to ' + schema_name + ';'

## 2. Utils functions
The function below allows to apply the inclusion criteria after the SQL query is done :
- The dataframe df obtained from the SQL query, contains some columns that are named "inclusion_...". These are boolean variables that take value `True` if the patient matches the criteria and should be included in the dataset.
- This function drops patients with any "inclusion\_..." set to `False` and then drops all the "inclusion\_..." variables because these are not useful features for the prediction

In [8]:
def apply_inclusion_criteria(df):
    print('Initial size of table : ' + str(df.shape[0]))
    df = df.drop_duplicates(['icustay_id','creatinine_time'])
    print('After dropping duplicates : ' + str(df.shape[0]))
    criteria_list = [i for i in df.columns.values if 'inclusion' in i]
    for c in criteria_list:
        df = df.loc[df[c]==1,:].drop(c,axis=1)    
    print('After applying inclusion criteria : ' + str(df.shape[0]))
    return df

## 3. Load file features_info.csv describing features to be found in chartevents table
- This table describes the list of item_id that should be used to retrieve a given variable in the chartevents table
- The list of item_id has been derived from the [d_items table](https://mimic.physionet.org/mimictables/d_items/)

In [9]:
# Read file
features_info = pd.read_csv('features_info.csv')
# Drop lines with no item_id
item_col = [c for c in features_info.columns.values if "item" in c]
features_info = features_info.dropna(axis=0, how='all', subset=item_col).reset_index(drop=True)
features_info.head()

,name,table,variable,item_id,item_id_2,item_id_3,item_id_4,item_id_5,item_id_6
0,Arterial_pressure_systolic,chartevents,valuenum,6.0,51.0,455.0,220050.0,220179.0,225309.0
1,Arterial_pressure_diastolic,chartevents,valuenum,8364.0,8368.0,8441.0,220051.0,220180.0,225310.0
2,Heart_rate,chartevents,valuenum,211.0,220045.0,NaN,NaN,NaN,NaN
3,Weight_daily,chartevents,valuenum,763.0,224639.0,NaN,NaN,NaN,NaN
4,Temperature,chartevents,valuenum,223761.0,678.0,645.0,NaN,NaN,NaN


## 4. Retrieve interesting features for patients that match inclusion criteria
Exclusion criteria : 
- Patients with age < 15 years
- Patients who stayed less than 48 hours in the ICU
- Patients under renal replacement therapy
- Patients suffering from ESDR
- Patients that had a kidney transplant

In [10]:
# From features_info, retrieve the list of item_id to use with chartevents
item_col = [c for c in features_info.columns.values if "item_id" in c] #column names of item_ids

#extract values of item_ids for Arterial_pressure_systolic and convert to string
item_list = features_info.loc[0,item_col].dropna().astype('int').values 
item_str = "(" + str(item_list[0])
for it in item_list[1:]:
    item_str = item_str + "," + str(it)
item_str = item_str + ")"

# List of ICD-9 codes to be excluded
icd_list = ['5856','V420','99681'] # ESRD: 5856 / kidney transplant: V420,99681
icd_str = "'{" + str(icd_list[0])
for icd in icd_list[1:]:
    icd_str = icd_str + "," + str(icd)
icd_str = icd_str + "}'"
#print(icd_str)

#first, we query the data and include only one feature from the features_info table, 
#later we will loop over these features in the features_info table
query = query_schema + """

with cr1 as
(
--select icustays x creatinine values within stays x diagnoses, excl. rrt
select
  --select basic icustay features
    icu.subject_id, icu.icustay_id, icu.intime, icu.outtime, EXTRACT(EPOCH FROM icu.outtime - icu.intime)/60.0/60.0 as length_of_stay,
    ce.valuenum as creatinine, ce.storetime as creatinine_time,
    EXTRACT('epoch' from icu.intime - pat.dob) / 60.0 / 60.0 / 24.0 / 365.242 AS age,
    (rrt.icustay_id is null) as rrt,
    diag.icd9_code as diagnosis 
  from icustays icu 
  --add creatinine values
  inner join chartevents ce
    on icu.subject_id = ce.subject_id
    and ce.itemid = 220615
    and ce.valuenum is not null
    and ce.storetime between icu.intime and icu.outtime 
  --add age, rrt status(if available) and diagnoses
  inner join patients pat
    ON icu.subject_id = pat.subject_id
  left outer join rrt 
    on icu.icustay_id = rrt.icustay_id
  inner join diagnoses_icd diag
    on icu.subject_id = diag.subject_id
    and icu.hadm_id = diag.hadm_id  
    --the above gives us 26k ICUstays with 150k creatinine values within the stays, 
    --2.9M rows in final data frame due to multiple diagnoses
),
cr as
(
--aggregates diagnoses into a string for each icustay x creatinine value, selects from table cr1 created above
select
    cr1.subject_id, cr1.icustay_id, cr1.intime, cr1.outtime,
    cr1.creatinine, cr1.creatinine_time,
    cr1.age,
    cr1.length_of_stay,
    cr1.rrt,
    --array_agg aggregates the diagnoses, so we do not get multiple lines due to multiple diagnoses
    array_agg(cr1.diagnosis) as diagnoses
  from cr1
  group by cr1.subject_id, cr1.icustay_id, cr1.intime, cr1.outtime, cr1.creatinine, cr1.creatinine_time, cr1.age,
    cr1.length_of_stay,cr1.rrt
),
cr_inc as
(
--compute inclusion criteria, selects from table cr created above
select
    cr.subject_id, cr.icustay_id, cr.intime, cr.outtime,
    cr.creatinine, cr.creatinine_time,
    cr.age, CASE
                WHEN cr.age >= 15 then 1
            ELSE 0 END
            as inclusion_age,
    cr.length_of_stay, CASE
                        WHEN cr.length_of_stay >= 48 then 1
                     ELSE 0 END
                     as inclusion_length_of_stay,
    cr.rrt, CASE
            WHEN cr.rrt = False then 1
          ELSE 0 END
          as inclusion_rrt,
    cr.diagnoses, CASE
            WHEN cr.diagnoses && """ + icd_str + """ then 0
          ELSE 1 END
          as inclusion_diagnoses
  from cr
  ),
cr_feat as
(
--add feature value, time and delay (time from feature time to creatinine time)
--selects from table cr_inc created above and from table original chartevents table ce
select
    cr_inc.subject_id, cr_inc.icustay_id, cr_inc.intime, cr_inc.outtime,
    cr_inc.creatinine, cr_inc.creatinine_time,
    cr_inc.age, cr_inc.inclusion_age,
    cr_inc.length_of_stay, cr_inc.inclusion_length_of_stay,
    cr_inc.rrt, cr_inc.inclusion_rrt, cr_inc.diagnoses, cr_inc.inclusion_diagnoses,
    ce.valuenum as """+features_info.loc[0,'name']+""", 
    EXTRACT('epoch' from cr_inc.creatinine_time - ce.storetime) as """+features_info.loc[0,'name']+"""_delay,
    ce.storetime as """+features_info.loc[0,'name']+"""_time,
    ce.itemid as """+features_info.loc[0,'name']+"""_itemid
  from cr_inc
  inner join """+features_info.loc[0,'table']+""" ce
    on cr_inc.subject_id = ce.subject_id
    and ce.itemid in """+ item_str +"""
    and ce."""+features_info.loc[0,'variable']+""" is not null
    and ce.storetime between cr_inc.intime and cr_inc.creatinine_time
)
--select from table cr_feat alias "a" created above
select 
    a.subject_id, a.icustay_id, a.intime, a.outtime,
    a.creatinine, a.creatinine_time,
    a.age, a.inclusion_age,
    a.length_of_stay, a.inclusion_length_of_stay,
    a.rrt, a.inclusion_rrt, a.diagnoses, a.inclusion_diagnoses,
    a."""+features_info.loc[0,'name']+""", a."""+features_info.loc[0,'name']+"""_delay,
    a."""+features_info.loc[0,'name']+"""_time,
    a."""+features_info.loc[0,'name']+"""_itemid
from cr_feat as a
    join (
        select creatinine_time, min("""+features_info.loc[0,'name']+"""_delay) as """+features_info.loc[0,'name']+"""_delay
        from cr_feat
        group by creatinine_time
    ) as b on a.creatinine_time = b.creatinine_time
where a."""+features_info.loc[0,'name']+"""_delay = b."""+features_info.loc[0,'name']+"""_delay
"""
df_chartevents = pd.read_sql_query(query, con)
df_chartevents = apply_inclusion_criteria(df_chartevents)
    
#now we do the same extraction process as before, but we loop over the features to get them all 
# Convert list of diagnoses into str (required to perform the merge)
df_chartevents.loc[:,'diagnoses'] = df_chartevents['diagnoses'].apply(lambda x: ', '.join(sorted(x)))

for i,row in features_info.loc[1:,:].iterrows():
    print('------------------------------------')
    print('--- Processing feature : ' + row['name'])
    # From features_info, retrieve the list of item_id to use with chartevents
    item_col = [c for c in features_info.columns.values if "item_id" in c]
    item_list = features_info.loc[i,item_col].dropna().astype('int').values
    item_str = "(" + str(item_list[0])
    for it in item_list[1:]:
        item_str = item_str + "," + str(it)
    item_str = item_str + ")"

    # List of ICD-9 codes to be excluded
    icd_list = ['5856','V420','99681'] # ESRD: 5856 / kidney transplant: V420,99681
    icd_str = "'{" + str(icd_list[0])
    for icd in icd_list[1:]:
        icd_str = icd_str + "," + str(icd)
    icd_str = icd_str + "}'"

    query = query_schema + """
    with cr1 as
    (
    --select icustays x creatinine values within stays x diagnoses, excl. rrt
    select
      --select basic icustay features
        icu.subject_id, icu.icustay_id, icu.intime, icu.outtime, EXTRACT(EPOCH FROM icu.outtime - icu.intime)/60.0/60.0 as length_of_stay,
        ce.valuenum as creatinine, ce.storetime as creatinine_time,
        EXTRACT('epoch' from icu.intime - pat.dob) / 60.0 / 60.0 / 24.0 / 365.242 AS age,
        (rrt.icustay_id is null) as rrt,
        diag.icd9_code as diagnosis 
      from icustays icu
      --add creatinine values
      inner join chartevents ce
        on icu.subject_id = ce.subject_id
        and ce.itemid = 220615
        and ce.valuenum is not null
        and ce.storetime between icu.intime and icu.outtime
      --add age, rrt status(if available) and diagnoses
      inner join patients pat
        ON icu.subject_id = pat.subject_id
      left outer join rrt 
        on icu.icustay_id = rrt.icustay_id
      inner join diagnoses_icd diag
        on icu.subject_id = diag.subject_id
        and icu.hadm_id = diag.hadm_id
    --the above gives us 26k ICUstays with 150k creatinine values within the stays, 
    --2.9M rows in final data frame due to multiple diagnoses
    ),
    cr as
    (
    --aggregates diagnoses into a string for each icustay x creatinine value, selects from table cr1 created above
    select
    cr1.subject_id, cr1.icustay_id, cr1.intime, cr1.outtime,
    cr1.creatinine, cr1.creatinine_time,
    cr1.age,
    cr1.length_of_stay,
    cr1.rrt,
    --array_agg aggregates the diagnoses, so we do not get multiple lines due to multiple diagnoses
    array_agg(cr1.diagnosis) as diagnoses
    from cr1
    group by cr1.subject_id, cr1.icustay_id, cr1.intime, cr1.outtime, cr1.creatinine, cr1.creatinine_time, cr1.age,
    cr1.length_of_stay,cr1.rrt
    ),
    cr_inc as
    (
    --compute inclusion criteria, selects from table cr created above
    select
    cr.subject_id, cr.icustay_id, cr.intime, cr.outtime,
        cr.creatinine, cr.creatinine_time,
        cr.age, CASE
                    WHEN cr.age >= 15 then 1
                ELSE 0 END
                as inclusion_age,
      cr.length_of_stay, CASE
                            WHEN cr.length_of_stay >= 48 then 1
                         ELSE 0 END
                         as inclusion_length_of_stay,
      cr.rrt, CASE
                WHEN cr.rrt = False then 1
              ELSE 0 END
              as inclusion_rrt,
      cr.diagnoses, CASE
                WHEN cr.diagnoses && """ + icd_str + """ then 0
              ELSE 1 END
              as inclusion_diagnoses
      from cr
    ),
    cr_feat as
    (
    --add feature value, time and delay (time from feature time to creatinine time)
    --selects from table cr_inc created above and from table original chartevents table ce
    select
    cr_inc.subject_id, cr_inc.icustay_id, cr_inc.intime, cr_inc.outtime,
        cr_inc.creatinine, cr_inc.creatinine_time,
        cr_inc.age, cr_inc.inclusion_age,
        cr_inc.length_of_stay, cr_inc.inclusion_length_of_stay,
        cr_inc.rrt, cr_inc.inclusion_rrt, cr_inc.diagnoses, cr_inc.inclusion_diagnoses,
        ce.valuenum as """+features_info.loc[i,'name']+""", 
        EXTRACT('epoch' from cr_inc.creatinine_time - ce.storetime) as """+features_info.loc[i,'name']+"""_delay,
        ce.storetime as """+features_info.loc[i,'name']+"""_time,
        ce.itemid as """+features_info.loc[i,'name']+"""_itemid
      from cr_inc
      inner join """+features_info.loc[i,'table']+""" ce
        on cr_inc.subject_id = ce.subject_id
        and ce.itemid in """+ item_str +"""
        and ce."""+features_info.loc[i,'variable']+""" is not null
        and ce.storetime between cr_inc.intime and cr_inc.creatinine_time
    )
    select 
    --select from table cr_feat alias "a" created above
    --why do we join on creatinine_time here?
    a.subject_id, a.icustay_id, a.intime, a.outtime,
        a.creatinine, a.creatinine_time,
        a.age, a.inclusion_age,
        a.length_of_stay, a.inclusion_length_of_stay,
        a.rrt, a.inclusion_rrt, a.diagnoses, a.inclusion_diagnoses,
        a."""+features_info.loc[i,'name']+""", a."""+features_info.loc[i,'name']+"""_delay,
        a."""+features_info.loc[i,'name']+"""_time,
        a."""+features_info.loc[i,'name']+"""_itemid
    from cr_feat as a
        join (
            select creatinine_time, min("""+features_info.loc[i,'name']+"""_delay) as """+features_info.loc[i,'name']+"""_delay
            from cr_feat
            group by creatinine_time
        ) as b on a.creatinine_time = b.creatinine_time
    where a."""+features_info.loc[i,'name']+"""_delay = b."""+features_info.loc[i,'name']+"""_delay
    """
    
    df = pd.read_sql_query(query, con)
    df = apply_inclusion_criteria(df)
    # Convert list of diagnoses into str (required to perform the merge)
    df.loc[:,'diagnoses'] = df['diagnoses'].apply(lambda x: ', '.join(sorted(x)))
    
    df_chartevents = pd.merge(df_chartevents,df,on=['subject_id', 'icustay_id', 'intime', 'outtime', 'creatinine',
       'creatinine_time', 'age', 'length_of_stay', 'rrt', 'diagnoses'],how='outer')
    print('Merged table size : ' + str(df_chartevents.shape[0]))
    print(df_chartevents.head())



Initial size of table : 208840
After dropping duplicates : 150528
After applying inclusion criteria : 114382
------------------------------------
--- Processing feature : Arterial_pressure_diastolic
Initial size of table : 208914
After dropping duplicates : 150526
After applying inclusion criteria : 114382
Merged table size : 114383
   subject_id  icustay_id              intime             outtime  creatinine  \
0        9096      289163 2160-03-25 14:29:46 2160-03-28 16:49:59         1.4   
1        9096      289163 2160-03-25 14:29:46 2160-03-28 16:49:59         1.1   
2       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         1.1   
3       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         0.9   
4       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         1.0   

      creatinine_time        age  length_of_stay    rrt  \
0 2160-03-26 21:56:00  55.698972       74.336944  False   
1 2160-03-27 08:43:00  55.698972       74.336944  False   
2

Initial size of table : 145880
After dropping duplicates : 142139
After applying inclusion criteria : 108923
Merged table size : 114694
   subject_id  icustay_id              intime             outtime  creatinine  \
0        9096      289163 2160-03-25 14:29:46 2160-03-28 16:49:59         1.4   
1        9096      289163 2160-03-25 14:29:46 2160-03-28 16:49:59         1.1   
2       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         1.1   
3       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         0.9   
4       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         1.0   

      creatinine_time        age  length_of_stay    rrt  \
0 2160-03-26 21:56:00  55.698972       74.336944  False   
1 2160-03-27 08:43:00  55.698972       74.336944  False   
2 2125-03-07 02:36:00  72.826190     1089.718333  False   
3 2125-03-16 00:36:00  72.826190     1089.718333  False   
4 2125-03-08 04:23:00  72.826190     1089.718333  False   

                      

Initial size of table : 19493
After dropping duplicates : 19487
After applying inclusion criteria : 15083
Merged table size : 114735
   subject_id  icustay_id              intime             outtime  creatinine  \
0        9096      289163 2160-03-25 14:29:46 2160-03-28 16:49:59         1.4   
1        9096      289163 2160-03-25 14:29:46 2160-03-28 16:49:59         1.1   
2       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         1.1   
3       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         0.9   
4       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         1.0   

      creatinine_time        age  length_of_stay    rrt  \
0 2160-03-26 21:56:00  55.698972       74.336944  False   
1 2160-03-27 08:43:00  55.698972       74.336944  False   
2 2125-03-07 02:36:00  72.826190     1089.718333  False   
3 2125-03-16 00:36:00  72.826190     1089.718333  False   
4 2125-03-08 04:23:00  72.826190     1089.718333  False   

                         

Initial size of table : 18364
After dropping duplicates : 18361
After applying inclusion criteria : 14480
Merged table size : 115044
   subject_id  icustay_id              intime             outtime  creatinine  \
0        9096      289163 2160-03-25 14:29:46 2160-03-28 16:49:59         1.4   
1        9096      289163 2160-03-25 14:29:46 2160-03-28 16:49:59         1.1   
2       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         1.1   
3       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         0.9   
4       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         1.0   

      creatinine_time        age  length_of_stay    rrt  \
0 2160-03-26 21:56:00  55.698972       74.336944  False   
1 2160-03-27 08:43:00  55.698972       74.336944  False   
2 2125-03-07 02:36:00  72.826190     1089.718333  False   
3 2125-03-16 00:36:00  72.826190     1089.718333  False   
4 2125-03-08 04:23:00  72.826190     1089.718333  False   

                         

Initial size of table : 19366
After dropping duplicates : 19364
After applying inclusion criteria : 15057
Merged table size : 115051
   subject_id  icustay_id              intime             outtime  creatinine  \
0        9096      289163 2160-03-25 14:29:46 2160-03-28 16:49:59         1.4   
1        9096      289163 2160-03-25 14:29:46 2160-03-28 16:49:59         1.1   
2       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         1.1   
3       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         0.9   
4       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         1.0   

      creatinine_time        age  length_of_stay    rrt  \
0 2160-03-26 21:56:00  55.698972       74.336944  False   
1 2160-03-27 08:43:00  55.698972       74.336944  False   
2 2125-03-07 02:36:00  72.826190     1089.718333  False   
3 2125-03-16 00:36:00  72.826190     1089.718333  False   
4 2125-03-08 04:23:00  72.826190     1089.718333  False   

                         

Initial size of table : 0
After dropping duplicates : 0
After applying inclusion criteria : 0
Merged table size : 115344
   subject_id  icustay_id              intime             outtime  creatinine  \
0        9096      289163 2160-03-25 14:29:46 2160-03-28 16:49:59         1.4   
1        9096      289163 2160-03-25 14:29:46 2160-03-28 16:49:59         1.1   
2       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         1.1   
3       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         0.9   
4       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         1.0   

      creatinine_time        age  length_of_stay    rrt  \
0 2160-03-26 21:56:00  55.698972       74.336944  False   
1 2160-03-27 08:43:00  55.698972       74.336944  False   
2 2125-03-07 02:36:00  72.826190     1089.718333  False   
3 2125-03-16 00:36:00  72.826190     1089.718333  False   
4 2125-03-08 04:23:00  72.826190     1089.718333  False   

                                     

Initial size of table : 6255
After dropping duplicates : 6255
After applying inclusion criteria : 5171
Merged table size : 115347
   subject_id  icustay_id              intime             outtime  creatinine  \
0        9096      289163 2160-03-25 14:29:46 2160-03-28 16:49:59         1.4   
1        9096      289163 2160-03-25 14:29:46 2160-03-28 16:49:59         1.1   
2       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         1.1   
3       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         0.9   
4       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         1.0   

      creatinine_time        age  length_of_stay    rrt  \
0 2160-03-26 21:56:00  55.698972       74.336944  False   
1 2160-03-27 08:43:00  55.698972       74.336944  False   
2 2125-03-07 02:36:00  72.826190     1089.718333  False   
3 2125-03-16 00:36:00  72.826190     1089.718333  False   
4 2125-03-08 04:23:00  72.826190     1089.718333  False   

                            

Initial size of table : 0
After dropping duplicates : 0
After applying inclusion criteria : 0
Merged table size : 115347
   subject_id  icustay_id              intime             outtime  creatinine  \
0        9096      289163 2160-03-25 14:29:46 2160-03-28 16:49:59         1.4   
1        9096      289163 2160-03-25 14:29:46 2160-03-28 16:49:59         1.1   
2       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         1.1   
3       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         0.9   
4       13806      215159 2125-02-05 04:24:09 2125-03-22 14:07:15         1.0   

      creatinine_time        age  length_of_stay    rrt  \
0 2160-03-26 21:56:00  55.698972       74.336944  False   
1 2160-03-27 08:43:00  55.698972       74.336944  False   
2 2125-03-07 02:36:00  72.826190     1089.718333  False   
3 2125-03-16 00:36:00  72.826190     1089.718333  False   
4 2125-03-08 04:23:00  72.826190     1089.718333  False   

                                     

In [11]:
# Dump to file
df_chartevents.to_csv('creatinine_measurements_1.csv')
df_chartevents.head()

,subject_id,icustay_id,intime,outtime,creatinine,creatinine_time,age,length_of_stay,rrt,diagnoses,...,total_protein_urine_time,total_protein_urine_itemid,bilirubin,bilirubin_delay,bilirubin_time,bilirubin_itemid,c_reactive_protein,c_reactive_protein_delay,c_reactive_protein_time,c_reactive_protein_itemid
0,9096,289163,2160-03-25 14:29:46,2160-03-28 16:49:59,1.4,2160-03-26 21:56:00,55.698972,74.336944,False,"42732, 42789, 5070, 5601, 56723, 5712, 5722, 5...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,9096,289163,2160-03-25 14:29:46,2160-03-28 16:49:59,1.1,2160-03-27 08:43:00,55.698972,74.336944,False,"42732, 42789, 5070, 5601, 56723, 5712, 5722, 5...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,13806,215159,2125-02-05 04:24:09,2125-03-22 14:07:15,1.1,2125-03-07 02:36:00,72.826190,1089.718333,False,"00845, 0389, 11289, 2449, 25000, 2720, 2851, 4...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,13806,215159,2125-02-05 04:24:09,2125-03-22 14:07:15,0.9,2125-03-16 00:36:00,72.826190,1089.718333,False,"00845, 0389, 11289, 2449, 25000, 2720, 2851, 4...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,13806,215159,2125-02-05 04:24:09,2125-03-22 14:07:15,1.0,2125-03-08 04:23:00,72.826190,1089.718333,False,"00845, 0389, 11289, 2449, 25000, 2720, 2851, 4...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


## 5. Retrieve missing static information
On top of the variables from chartevents table, retrieve following static information that may have an impact on the quality of the predictions :
- Reason of stay (i.e. diagnosis at entrance)
- Ethnicity
- Age
- Gender

In [12]:
query = query_schema + """
with cr1 as
(
select
    icu.subject_id, icu.icustay_id, icu.intime, icu.outtime, EXTRACT(EPOCH FROM icu.outtime - icu.intime)/60.0/60.0 as length_of_stay,
    ce.valuenum as creatinine, ce.storetime as creatinine_time,
    adm.ethnicity, adm.diagnosis as diagnosis,
    pat.gender as gender,
    EXTRACT('epoch' from icu.intime - pat.dob) / 60.0 / 60.0 / 24.0 / 365.242 AS age,
    (rrt.icustay_id is null) as rrt,
    diag.icd9_code as diag 
  from icustays icu
  inner join chartevents ce
    on icu.subject_id = ce.subject_id
    and ce.itemid = 220615
    and ce.valuenum is not null
    and ce.storetime between icu.intime and icu.outtime
  inner join patients pat
    ON icu.subject_id = pat.subject_id
  inner join admissions adm
    on icu.subject_id = adm.subject_id
  left outer join rrt 
    on icu.icustay_id = rrt.icustay_id
  inner join diagnoses_icd diag
    on icu.subject_id = diag.subject_id
    and icu.hadm_id = diag.hadm_id
),
cr as
(
select
cr1.subject_id, cr1.icustay_id, cr1.intime, cr1.outtime,
cr1.creatinine, cr1.creatinine_time,
cr1.ethnicity, cr1.diagnosis,
cr1.gender as gender,
cr1.age,
cr1.length_of_stay,
cr1.rrt,
array_agg(cr1.diag) as diagnoses
from cr1
group by cr1.subject_id, cr1.icustay_id, cr1.intime, cr1.outtime, cr1.creatinine, cr1.creatinine_time,
cr1.ethnicity, cr1.diagnosis, cr1.gender, cr1.age, cr1.length_of_stay, cr1.rrt
)
select
cr.subject_id, cr.icustay_id, cr.intime, cr.outtime,
    cr.creatinine, cr.creatinine_time,
    cr.ethnicity, cr.diagnosis, cr.gender,
    cr.age, CASE
                WHEN cr.age >= 15 then 1
            ELSE 0 END
            as inclusion_age,
  cr.length_of_stay, CASE
                        WHEN cr.length_of_stay >= 48 then 1
                     ELSE 0 END
                     as inclusion_length_of_stay,
  cr.rrt, CASE
            WHEN cr.rrt = False then 1
          ELSE 0 END
          as inclusion_rrt,
  cr.diagnoses, CASE
            WHEN cr.diagnoses && """ + icd_str + """ then 0
          ELSE 1 END
          as inclusion_diagnoses
  from cr
"""
df_static = pd.read_sql_query(query, con)
df_static = apply_inclusion_criteria(df_static)

# Convert list of diagnoses into str (required to perform the merge)
df_static.loc[:,'diagnoses'] = df_static['diagnoses'].apply(lambda x: ', '.join(sorted(x)))
print(df_static.head())

Initial size of table : 288243
After dropping duplicates : 152630
After applying inclusion criteria : 115784
   subject_id  icustay_id              intime             outtime  creatinine  \
0       96260      278161 2196-08-31 18:24:20 2196-09-05 15:39:35         1.1   
1       92788      241776 2148-12-05 18:00:11 2148-12-23 19:52:48         5.8   
3       56854      279535 2201-02-02 11:39:44 2201-03-17 14:55:55         1.8   
4       74626      274172 2137-05-06 20:20:31 2137-05-30 15:14:22         1.3   
6       90834      280700 2161-05-16 07:30:53 2161-05-20 19:03:29         0.7   

      creatinine_time               ethnicity                 diagnosis  \
0 2196-08-31 21:23:00  BLACK/AFRICAN AMERICAN  STEVENS JOHNSON SYNDROME   
1 2148-12-09 06:13:00                   WHITE              ENDOCARDITIS   
3 2201-02-04 01:46:00                   WHITE               HYPOTENSION   
4 2137-05-17 05:09:00                   WHITE                   DYSPNEA   
6 2161-05-20 16:27:00        

In [13]:
# Dump to file
df_static.to_csv('creatinine_measurements_2.csv')
print(df_static)

        subject_id  icustay_id              intime             outtime  \
0            96260      278161 2196-08-31 18:24:20 2196-09-05 15:39:35   
1            92788      241776 2148-12-05 18:00:11 2148-12-23 19:52:48   
3            56854      279535 2201-02-02 11:39:44 2201-03-17 14:55:55   
4            74626      274172 2137-05-06 20:20:31 2137-05-30 15:14:22   
6            90834      280700 2161-05-16 07:30:53 2161-05-20 19:03:29   
7            31779      217318 2105-12-10 04:47:35 2105-12-30 17:02:55   
8            15311      267389 2148-12-28 17:02:35 2148-12-31 13:09:18   
9            73440      203714 2161-11-23 00:55:11 2161-11-25 13:29:43   
10           77484      200282 2164-05-03 02:49:44 2164-05-18 13:47:57   
11           57935      287815 2146-08-19 21:14:03 2146-08-31 18:48:59   
13           90369      235667 2149-11-04 16:38:12 2149-11-11 11:22:02   
14           29968      202124 2124-12-12 18:11:02 2125-01-06 18:40:01   
15           91046      265671 2156-10

## 6. Merge df_chartevents and df_static tables into one
Create one unique dataframe holding the static and dynamic features

In [14]:
print('Chartevents :')
print(df_chartevents.shape)
print(df_chartevents.columns.values)
print('')

print('Static:')
print(df_static.shape)
print(df_static.columns.values)

Chartevents :
(115347, 118)
['subject_id' 'icustay_id' 'intime' 'outtime' 'creatinine'
 'creatinine_time' 'age' 'length_of_stay' 'rrt' 'diagnoses'
 'arterial_pressure_systolic' 'arterial_pressure_systolic_delay'
 'arterial_pressure_systolic_time' 'arterial_pressure_systolic_itemid'
 'arterial_pressure_diastolic' 'arterial_pressure_diastolic_delay'
 'arterial_pressure_diastolic_time' 'arterial_pressure_diastolic_itemid'
 'heart_rate' 'heart_rate_delay' 'heart_rate_time' 'heart_rate_itemid'
 'weight_daily' 'weight_daily_delay' 'weight_daily_time'
 'weight_daily_itemid' 'temperature' 'temperature_delay'
 'temperature_time' 'temperature_itemid' 'urine_output'
 'urine_output_delay' 'urine_output_time' 'urine_output_itemid'
 'day_urine_output' 'day_urine_output_delay' 'day_urine_output_time'
 'day_urine_output_itemid' 'scr' 'scr_delay' 'scr_time' 'scr_itemid'
 'sodium' 'sodium_delay' 'sodium_time' 'sodium_itemid' 'potassium'
 'potassium_delay' 'potassium_time' 'potassium_itemid' 'calcium'
 '

In [15]:
merged_df = pd.merge(df_chartevents,df_static,on=['subject_id', 'icustay_id', 'intime', 'outtime', 'creatinine',
       'creatinine_time', 'age', 'length_of_stay', 'rrt', 'diagnoses'],how='outer')

## 7. Remove columns that are not features 
(except icustay_id and patient_id that are needed to build the table with labels)

In [16]:
# REMOVE COLUMNS THAT WERE USED FOR INCLUSION CRITERIA BUT THAT ARE NOT AVAILABLE FEATURES FOR THE PREDICTION
# REMOVE ICUSTAY_IDS FOR WHICH THERE IS ONLY ONE MEASUREMENT OF CREATININE

print('Number of lines : ' + str(merged_df.shape[0]))
to_remove = ['intime','outtime','length_of_stay','rrt','diagnoses']
for c in to_remove:
    if (c in merged_df.columns.values): merged_df = merged_df.drop(c,axis=1)
        
# Remove columns with _time suffix EXCEPT the time for creatinine which is required to compute the labels
to_remove = [i for i in merged_df.columns.values if (('_time' in i) & (i!='creatinine_time'))]
for c in to_remove:
    if (c in merged_df.columns.values): merged_df = merged_df.drop(c,axis=1)

# Remove columns with _itemid suffix
to_remove = [i for i in merged_df.columns.values if '_itemid' in i]
for c in to_remove:
    if (c in merged_df.columns.values): merged_df = merged_df.drop(c,axis=1)

# Remove icustay_ids for which there's only one measurement of creatinine
count_mes = merged_df['icustay_id'].value_counts()
to_remove = count_mes.index.values[count_mes==1]
for i in to_remove:
    merged_df = merged_df.loc[merged_df['icustay_id']!=i,:]

# Drop duplicate values of creatinine_time
merged_df = merged_df.drop_duplicates(['icustay_id','creatinine_time'])
print('After dropping unique measurements of creatinine : ' + str(merged_df.shape[0]))
merged_df.head()

Number of lines : 119105
After dropping unique measurements of creatinine : 115717


,subject_id,icustay_id,creatinine,creatinine_time,age,arterial_pressure_systolic,arterial_pressure_systolic_delay,arterial_pressure_diastolic,arterial_pressure_diastolic_delay,heart_rate,...,albumin_delay,total_protein_urine,total_protein_urine_delay,bilirubin,bilirubin_delay,c_reactive_protein,c_reactive_protein_delay,ethnicity,diagnosis,gender
0,9096,289163,1.4,2160-03-26 21:56:00,55.698972,114.0,1740.0,52.0,1740.0,100.0,...,62280.0,NaN,NaN,NaN,NaN,NaN,NaN,WHITE,FEVER,M
1,9096,289163,1.1,2160-03-27 08:43:00,55.698972,136.0,2520.0,59.0,2520.0,107.0,...,101100.0,NaN,NaN,NaN,NaN,NaN,NaN,WHITE,FEVER,M
2,13806,215159,1.1,2125-03-07 02:36:00,72.826190,110.0,1440.0,41.0,1440.0,66.0,...,510600.0,NaN,NaN,NaN,NaN,NaN,NaN,WHITE,FEVER;RULE-OUT MYOCARDIAL INFARCTION;TELEMETRY,F
3,13806,215159,0.9,2125-03-16 00:36:00,72.826190,104.0,1980.0,52.0,1980.0,110.0,...,565260.0,NaN,NaN,NaN,NaN,NaN,NaN,WHITE,GROIN BLEED;CP;TELEMETRY,F
4,13806,215159,1.0,2125-03-08 04:23:00,72.826190,98.0,6240.0,44.0,6240.0,83.0,...,603420.0,NaN,NaN,NaN,NaN,NaN,NaN,WHITE,FEVER;RULE-OUT MYOCARDIAL INFARCTION;TELEMETRY,F


**WARNING : the columns named "..._itemid" can be used to filter on the itemid used to retrieve the feature after the query has been done. But these are not features.**

In [17]:
# Dump to file
merged_df.to_csv('creatinine_measurements_merged.csv')
print(merged_df)

        subject_id  icustay_id  creatinine     creatinine_time         age  \
0             9096      289163         1.4 2160-03-26 21:56:00   55.698972   
1             9096      289163         1.1 2160-03-27 08:43:00   55.698972   
2            13806      215159         1.1 2125-03-07 02:36:00   72.826190   
3            13806      215159         0.9 2125-03-16 00:36:00   72.826190   
4            13806      215159         1.0 2125-03-08 04:23:00   72.826190   
5            13806      215159         0.9 2125-03-15 04:07:00   72.826190   
6            13806      215159         1.0 2125-03-17 05:22:00   72.826190   
7            13806      215159         0.8 2125-03-14 03:36:00   72.826190   
8            13806      215159         0.9 2125-03-15 16:52:00   72.826190   
9            13806      215159         1.9 2125-02-19 04:37:00   72.826190   
10           13806      215159         1.4 2125-02-25 00:32:00   72.826190   
11           16784      267119         0.8 2137-04-13 04:53:00  

## 8. Create labels for evolution of creatinine
- The models would be used around 6a.m., when the team for the day begins to work 
- The prediction would concern the values of creatinine the next day (how it will have evolved after 20-24 hours)
- Therefore, we take into account only measurements of creatinine performed between midnight and 5am, and consider the "next day" is "next day between midnight and 5am"

In [18]:
merged_df = pd.read_csv('creatinine_measurements_merged.csv').drop('Unnamed: 0',axis=1)

In [19]:
# Keep only creatinine measured between 0am and 5am
merged_df.loc[:,'creatinine_hour'] = merged_df['creatinine_time'].astype('datetime64[ns]').apply(lambda x: x.hour)
merged_df = merged_df.loc[((merged_df['creatinine_hour']>=0) & (merged_df['creatinine_hour']<=5)) ,:]

In [20]:
# In each row, report values of creatinine for next day and days before
merged_df.loc[:,'creatinine_time'] = merged_df.loc[:,'creatinine_time'].astype('datetime64[ns]')
for i, row in merged_df.iterrows():
    same_stay = merged_df.loc[merged_df['icustay_id']==row['icustay_id'],:]
    delay = (same_stay['creatinine_time']-row['creatinine_time']).apply(lambda x: x.days)
    merged_df.loc[i,'creatinine_tomorrow'] =same_stay.loc[delay==1,'creatinine'].max()
    merged_df.loc[i,'creatinine_yesterday'] =same_stay.loc[delay==-1,'creatinine'].max()
    merged_df.loc[i,'creatinine_before_yesterday'] =same_stay.loc[delay==-2,'creatinine'].max()

In [21]:
def create_labels(diff):
    if diff>0.15 :
        return 0
    elif diff<-0.15 :
        return 1
    elif math.isnan(diff):
        return None
    else:
        return 2

merged_df.loc[:,'creatinine_diff'] = merged_df['creatinine_tomorrow']-merged_df['creatinine']
merged_df.loc[:,'label'] = merged_df['creatinine_diff'].apply(create_labels)

# Drop NaN labels (lines with no "creatinine_tomorrow")
merged_df = merged_df.dropna(subset=['label'])
print(merged_df)

        subject_id  icustay_id  creatinine     creatinine_time         age  \
2            13806      215159         1.1 2125-03-07 02:36:00   72.826190   
3            13806      215159         0.9 2125-03-16 00:36:00   72.826190   
4            13806      215159         1.0 2125-03-08 04:23:00   72.826190   
6            13806      215159         1.0 2125-03-17 05:22:00   72.826190   
7            13806      215159         0.8 2125-03-14 03:36:00   72.826190   
11           16784      267119         0.8 2137-04-13 04:53:00   77.900075   
18           27192      265950         1.0 2171-04-14 04:19:00   54.879741   
21           27192      265950         0.9 2171-04-12 02:03:00   54.879741   
22           27856      285406         1.3 2103-03-01 04:39:00   79.198437   
24           28045      213323         0.9 2141-01-02 05:26:00   69.150468   
25           28247      269523         0.9 2167-06-27 04:28:00   24.263434   
28           28247      269523         0.8 2167-06-28 03:03:00  

In [22]:
print(merged_df)

        subject_id  icustay_id  creatinine     creatinine_time         age  \
2            13806      215159         1.1 2125-03-07 02:36:00   72.826190   
3            13806      215159         0.9 2125-03-16 00:36:00   72.826190   
4            13806      215159         1.0 2125-03-08 04:23:00   72.826190   
6            13806      215159         1.0 2125-03-17 05:22:00   72.826190   
7            13806      215159         0.8 2125-03-14 03:36:00   72.826190   
11           16784      267119         0.8 2137-04-13 04:53:00   77.900075   
18           27192      265950         1.0 2171-04-14 04:19:00   54.879741   
21           27192      265950         0.9 2171-04-12 02:03:00   54.879741   
22           27856      285406         1.3 2103-03-01 04:39:00   79.198437   
24           28045      213323         0.9 2141-01-02 05:26:00   69.150468   
25           28247      269523         0.9 2167-06-27 04:28:00   24.263434   
28           28247      269523         0.8 2167-06-28 03:03:00  

In [23]:
# Remove features that are not labels
to_remove = ['creatinine_time','creatinine_hour','creatinine_tomorrow','creatinine_diff']
for c in to_remove:
    if (c in merged_df.columns.values): merged_df = merged_df.drop(c,axis=1)
print(merged_df.columns)

Index(['subject_id', 'icustay_id', 'creatinine', 'age',
       'arterial_pressure_systolic', 'arterial_pressure_systolic_delay',
       'arterial_pressure_diastolic', 'arterial_pressure_diastolic_delay',
       'heart_rate', 'heart_rate_delay', 'weight_daily', 'weight_daily_delay',
       'temperature', 'temperature_delay', 'urine_output',
       'urine_output_delay', 'day_urine_output', 'day_urine_output_delay',
       'scr', 'scr_delay', 'sodium', 'sodium_delay', 'potassium',
       'potassium_delay', 'calcium', 'calcium_delay', 'phosphor',
       'phosphor_delay', 'hemoglobine', 'hemoglobine_delay', 'uric_acid',
       'uric_acid_delay', 'chloride', 'chloride_delay', 'platelet_count',
       'platelet_count_delay', 'ph_blood', 'ph_blood_delay', 'fibrinogen',
       'fibrinogen_delay', 'urinary_sodium', 'urinary_sodium_delay',
       'urinary_potassium', 'urinary_potassium_delay', 'urine_creatinin',
       'urine_creatinin_delay', 'alkaline_phospatase',
       'alkaline_phospatase_de

In [24]:
# Dump to file
merged_df.to_csv('dataset_with_labels.csv')
print(merged_df)

        subject_id  icustay_id  creatinine         age  \
2            13806      215159         1.1   72.826190   
3            13806      215159         0.9   72.826190   
4            13806      215159         1.0   72.826190   
6            13806      215159         1.0   72.826190   
7            13806      215159         0.8   72.826190   
11           16784      267119         0.8   77.900075   
18           27192      265950         1.0   54.879741   
21           27192      265950         0.9   54.879741   
22           27856      285406         1.3   79.198437   
24           28045      213323         0.9   69.150468   
25           28247      269523         0.9   24.263434   
28           28247      269523         0.8   24.263434   
32           28528      270380         1.2   46.427915   
33           28528      270380         0.7   46.427915   
34           28758      285392         1.2   35.077751   
35           28815      202164         3.7  300.028712   
36           2